In [1]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor  , AutoModelForImageClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from PIL import Image
import torch

c:\Users\mtlun\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Loading FER-2013 Dataset:  https://huggingface.co/datasets/3una/Fer2013

dataset = load_dataset("Piro17/dataset-affecthqnet-fer2013")

sample_train = 22000
sample_test = 3000
seed = 27
print(dataset)

dataset['train'] = dataset['train'].shuffle(seed=seed).select(range(sample_train))
dataset['test'] = dataset['train'].shuffle(seed=seed).select(range(sample_test))

test_valid_split = dataset['test'].train_test_split(test_size=0.65, seed=45)
dataset['test'] = test_valid_split['train']
dataset['validation'] = test_valid_split['test']

#Import ResNet-50 from HuggingFace
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-26")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-26")


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 56532
    })
})


c:\Users\mtlun\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [5]:
print(dataset['train'][0])

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=48x48 at 0x1EAD2D66610>, 'label': 4}


In [6]:
data_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(size=(48, 48), scale=(0.8, 1.0)),
    transforms.ToTensor()
])

In [7]:
# Define the transform function
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([img.convert("RGB") for img in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

# Apply the transform to the datasets
dataset = dataset.map(transform, batched=True)

# Remove the 'image' column as it's now transformed
dataset = dataset.remove_columns(['image'])

# Set the format for PyTorch
dataset.set_format(type='torch')

Map: 100%|██████████| 1950/1950 [01:46<00:00, 18.32 examples/s]


In [8]:
from transformers import Trainer

In [9]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [10]:
from transformers import EarlyStoppingCallback

In [13]:
#Training Args
training_args = TrainingArguments(
    output_dir='./huggingface_fer_model/results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,   # batch size for evaluation
    evaluation_strategy="epoch",     # evaluation strategy to use at the end of each epoch
    save_strategy="epoch",           # save strategy to use at the end of each epoch
    logging_dir='./huggingface_fer_model/logs',            # directory for storing logs
    logging_steps=25,
    warmup_steps=55,                 # number of warmup steps for learning rate scheduler
    report_to=[],                    # disable reporting to any integration
    learning_rate=7e-5,
    weight_decay=0.055,
    fp16=True,                     # use mixed precision training
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    metric_for_best_model="eval_loss",
    greater_is_better=False,          # lower loss is better
    save_total_limit=2,               # limit the total amount of checkpoints, delete the older checkpoints in the output_dir    
)

#Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=dataset['train'],      
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5) ]  # Stop training if no improvement
)

In [ ]:
trainer.train()

                                                     
  1%|          | 233/22925 [02:14<2:54:06,  2.17it/s]

{'loss': 6.0068, 'grad_norm': 25.996702194213867, 'learning_rate': 2.8e-05, 'epoch': 0.01}


                                                     
  1%|          | 233/22925 [02:17<2:54:06,  2.17it/s]

{'loss': 5.1254, 'grad_norm': 29.352420806884766, 'learning_rate': 5.981818181818181e-05, 'epoch': 0.02}


                                                     
  1%|          | 233/22925 [02:20<2:54:06,  2.17it/s]

...

{'loss': 1.0949, 'grad_norm': 10.821778297424316, 'learning_rate': 6.562324649298597e-05, 'epoch': 0.65}


                                                     ]
  1%|          | 233/22925 [07:07<2:54:06,  2.17it/s] 

{'loss': 1.0102, 'grad_norm': 16.203832626342773, 'learning_rate': 6.555948260156677e-05, 'epoch': 0.65}


                                                     
  1%|          | 233/22925 [07:12<2:54:06,  2.17it/s] 

{'loss': 1.0877, 'grad_norm': 15.98011302947998, 'learning_rate': 6.549571871014756e-05, 'epoch': 0.66}


In [ ]:
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Validation Precision: {eval_results['eval_precision']:.4f}")
print(f"Validation Recall: {eval_results['eval_recall']:.4f}")
print(f"Validation F1 Score: {eval_results['eval_f1']:.4f}")

Validation Loss: 2.7327
Validation Accuracy: 0.6508
Validation Precision: 0.6494
Validation Recall: 0.6508
Validation F1 Score: 0.6492


In [ ]:
model.save_pretrained('./resnet26_fer2013_model')
feature_extractor.save_pretrained('./resnet26_fer2013_model')

['./resnet26_fer2013_model/preprocessor_config.json']